In [ ]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import pyglet
import argparse
import imutils
import time
import dlib
import cv2
import pygame

import matplotlib.pyplot as plt

%matplotlib inline

pygame.mixer.init()

In [ ]:
p = "../deep/computer_vision/TirednessDetection/core/metal_gear.wav"

In [ ]:
def sound_alarm(path=p):
    pygame.mixer.music.load(path)
    pygame.mixer.music.play()

In [ ]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])

    return (A + B) / (2.0 * C)

In [ ]:
eyes_closed_tresh = 0.3
frames_closed_tresh = 10

In [ ]:
frames_closed = 0
alarm = False

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../models/shape_predictor_68_face_landmarks.dat")

In [ ]:
(left_s, left_e) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(right_s, right_e) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [ ]:
def visualize_eyes(left, right, frame):
    left_eye_hull = cv2.convexHull(left)
    right_eye_hull = cv2.convexHull(right)
    cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1)
    cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)

In [ ]:
def get_eyes(face, gray):
    shape = predictor(gray, face)
    shape = face_utils.shape_to_np(shape)
    
    return shape[left_s:left_e], shape[right_s:right_e]

In [ ]:
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 0)
    for face in faces:
        left_eyes, right_eyes = get_eyes(face, gray)
        visualize_eyes(left_eyes, right_eyes, frame)
        lratio = eye_aspect_ratio(left_eyes)
        rratio = eye_aspect_ratio(right_eyes)
        if lratio < eyes_closed_tresh and rratio < eyes_closed_tresh:
            frames_closed += 1
            if frames_closed >= frames_closed_tresh:
                Thread(target=sound_alarm).start()
                cv2.putText(frame, "ALERT!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            frames_closed = 0
        cv2.putText(frame, "Opened left_eyes: {:.2f} \n Opened right_eyes: {:.2f}".format(lratio, rratio), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow("Frame", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()  
     